## TODO
* Features:
    * Easy visualization of: case starts, concurrent case obligations
    * printouts/reports of parameters used to generate analyses
    * utils: determine if an event falls in an arbitrary time window; aggregate to show a weekly incidence (across days of week), vs 52 weeks/yr, 12 months/year; expected formats for activity events 
    * some clinical questions to be answered: mishaps during nights and wkends vs weekdays, per provider, or likelihood when activity (anesthesia demand) is high
* tests:
    * make an ActivityDataset from valid data collection
    * make an ActivityDataset from valid DataFrame
    * proceess concurrency from valid ActivityDataset

## Done
* subclass of DataFrame for AnesthesiaData
* subclass of AnethesiaData for AnesthesiaCaseData
* subclass of AnesthesiaData for AnesthesiaCaseEventData
* subclass of AnesthesiaData for AnesthesiaCaseMedicationData
* subclass of AnesthesiaData for AnesthesiaCaseStaffingData
* object for AnesthesiaCase
* object for AnesthesiaCaseEvent
* object for AnesthesiaCaseMedication
* object for AnestheisaCaseStaff

## Next
* helper function(s) to crank through cases, events, meds, and staffing, and create a collection of AnesthesiaCase objects with associated events and meds

In [1]:
import sys
sys.path.append("/home/tj/PycharmProjects/pytiva")

import os
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm import tqdm

import pytiva

In [2]:
wd = r'/home/tj/PycharmProjects/pytiva/tests/test_data'
filename = 'OBCapacity_Output.xlsx'

df_meds = pd.read_excel(os.path.join(wd, filename), sheet_name='Meds')

meds_column_map = {
    # will include only these momentarily
    'PAT_ENC_CSN_ID': 'case_id',
    'MED_DISPLAY_NAME': 'med_label',
    'MED_DESC': 'med_desc',
    'ORDER_DATETIME': 'med_datetime',
    'MAR_ACTION': 'med_action',
    'ORDER_STATUS': 'med_status'
}

df_meds_slim = df_meds.rename(columns=meds_column_map)[ meds_column_map.values() ]
med_ds = pytiva.anesthesia.AnesthesiaCaseMedicationsDataSet(df_meds_slim)
med_ds

,case_id,med_label,med_desc,med_datetime,med_action,med_status
0,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued
1,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued
2,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:44:00,Given,Completed
3,198075089,bupivacaine 0.25% (MARCAINE PF) injection,BUPIVACAINE (PF) 0.25 % (2.5 MG/ML) INJECTION ...,2021-01-04 23:44:00,Given,Completed
4,198075089,"lidocaine-epinephrine 2 %-1:200,000 injection","LIDOCAINE-EPINEPHRINE (PF) 2 %-1:200,000 INJEC...",2021-01-04 23:50:00,Given,Discontinued
...,...,...,...,...,...,...
9507,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9508,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9509,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9510,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued


In [9]:
df_meds_slim.to_csv(os.path.join(wd, 'med_test_data.csv'), index=False)

In [13]:
df_meds = pd.read_csv(os.path.join(wd, 'med_test_data.csv'))
med_ds = pytiva.anesthesia.AnesthesiaCaseMedicationsDataSet(df_meds_slim)
med_ds

,case_id,med_label,med_desc,med_datetime,med_action,med_status
0,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued
1,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued
2,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:44:00,Given,Completed
3,198075089,bupivacaine 0.25% (MARCAINE PF) injection,BUPIVACAINE (PF) 0.25 % (2.5 MG/ML) INJECTION ...,2021-01-04 23:44:00,Given,Completed
4,198075089,"lidocaine-epinephrine 2 %-1:200,000 injection","LIDOCAINE-EPINEPHRINE (PF) 2 %-1:200,000 INJEC...",2021-01-04 23:50:00,Given,Discontinued
...,...,...,...,...,...,...
9507,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9508,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9509,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9510,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued


## TODO: obfuscate and set aside med test data

In [14]:
pytiva.utils.hash_cols_in_df(med_ds, pytiva.utils.hash_and_slash, cols=['med_label'])

,case_id,med_label,med_desc,med_datetime,med_action,med_status
0,198075089,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued
1,198075089,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued
2,198075089,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:44:00,Given,Completed
3,198075089,487dc97cb0c16ea6afa887c97be3314d78279ff281766b...,BUPIVACAINE (PF) 0.25 % (2.5 MG/ML) INJECTION ...,2021-01-04 23:44:00,Given,Completed
4,198075089,1a9247d02bfa4b5e5567fbb9fe81262053543a5da74127...,"LIDOCAINE-EPINEPHRINE (PF) 2 %-1:200,000 INJEC...",2021-01-04 23:50:00,Given,Discontinued
...,...,...,...,...,...,...
9507,201377906,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9508,201377906,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9509,201377906,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued
9510,201377906,ba0ce33582b9b522573b16f04e479841d873830edddacf...,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued


In [15]:
three_minutes = pd.to_timedelta(3, unit='Min')
ads = med_ds.to_activity_dataset(offset_before=three_minutes, offset_after=three_minutes)
ads

,case_id,med_label,med_desc,med_datetime,med_action,med_status,activity_start,activity_end,activity
0,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued,2021-01-04 23:39:00,2021-01-04 23:45:00,medication
1,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:42:00,Given,Discontinued,2021-01-04 23:39:00,2021-01-04 23:45:00,medication
2,198075089,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-01-04 23:44:00,Given,Completed,2021-01-04 23:41:00,2021-01-04 23:47:00,medication
3,198075089,bupivacaine 0.25% (MARCAINE PF) injection,BUPIVACAINE (PF) 0.25 % (2.5 MG/ML) INJECTION ...,2021-01-04 23:44:00,Given,Completed,2021-01-04 23:41:00,2021-01-04 23:47:00,medication
4,198075089,"lidocaine-epinephrine 2 %-1:200,000 injection","LIDOCAINE-EPINEPHRINE (PF) 2 %-1:200,000 INJEC...",2021-01-04 23:50:00,Given,Discontinued,2021-01-04 23:47:00,2021-01-04 23:53:00,medication
...,...,...,...,...,...,...,...,...,...
9507,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued,2021-02-01 12:50:00,2021-02-01 12:56:00,medication
9508,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued,2021-02-01 12:50:00,2021-02-01 12:56:00,medication
9509,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued,2021-02-01 12:50:00,2021-02-01 12:56:00,medication
9510,201377906,fentaNYL PF (SUBLIMAZE) injection,FENTANYL (PF) 50 MCG/ML INJECTION SOLUTION,2021-02-01 12:53:00,Given,Discontinued,2021-02-01 12:50:00,2021-02-01 12:56:00,medication


In [21]:
for td in (pd.to_timedelta(n, unit='Min') for n in range(-3, 3)):
    test_ads = med_ds.to_activity_dataset(offset_before=td, offset_after=td)
    print(all(test_ads['activity_end'] - test_ads['activity_start'] == 2 * td))

True
True
True
True
True
True


In [16]:
strata_cols = ['case_id']#, 'med_label']
big_df = ads.fetch_unduplicated_concurrency(activities=['medication'], strata=strata_cols)

100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 431.73it/s]


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 321.62it/s]


100%|████████████████████████████████████████████| 8/8 [00:00<00:00, 389.26it/s]


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 282.92it/s]


100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 310.27it/s]


...

100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 645.82it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 309.05it/s]


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 274.72it/s]


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 327.32it/s]


100%|██████████████████████████████████████████| 20/20 [00:00<00:00, 369.73it/s]


In [22]:
big_df

,activity_start,activity_end,activity
0,2021-01-04 23:39:00,2021-01-05 00:00:00,activity
1,2021-01-05 00:14:00,2021-01-05 00:20:00,activity
2,2021-01-05 00:37:00,2021-01-05 00:43:00,activity
3,2021-01-05 00:50:00,2021-01-05 00:56:00,activity
4,2020-12-30 21:16:00,2020-12-30 21:22:00,activity
...,...,...,...
2689,2021-02-01 10:18:00,2021-02-01 10:24:00,activity
2690,2021-02-01 11:52:00,2021-02-01 11:58:00,activity
2691,2021-02-01 12:50:00,2021-02-01 12:56:00,activity
2692,2021-02-01 13:09:00,2021-02-01 13:15:00,activity


In [17]:
big_df.to_csv(os.path.join(wd, 'ads_fetch_unduplicated_concurrency_out.csv'), index=False)

In [ ]:
ads._unduplicated_concurrency_to_df(pytiva.ActivityDataSet(sample))

In [ ]:
strata_cols = ['case_id', 'med_label']
for g in ads._generate_stratified_df_slices(strata_cols)
    print(g)

In [ ]:
# this is for doing groupby-style slices and filtering and shit
fd = pytiva.activity.utils.col_val_combinations_to_dicts(ads, ['case_id'])

def y_fd(fd):
    for f in fd:
        df_slice = ads

        # this next loop is a strong candidate for parallelization
        for k, v in f.items():
            df_slice = df_slice[ df_slice[k] == v ]
            yield df_slice

In [ ]:
i = y_fd(fd)
i

In [ ]:
for j in y_fd(fd):
    if len(j):
        print(j)

In [ ]:
three_minutes = pd.to_timedelta(3, unit='Min')
results = []

# TODO: finish making a way to loop through all the per person shit and condense it into a single timeline of activity

def unduplicated_activity(activity_dataset):
    
    pass

for c in med_ds['case_id'].unique():#[:5]:
    mwts = med_ds._with_timespans(td_offset_after=three_minutes, td_offset_before=three_minutes)
    mwts['activity'] = 'medication'
    ads = pytiva.ActivityDataSet(mwts[ mwts['case_id'].isin([c]) ])
    unduplicated = ads.unduplicated_concurrency_ads()._df
    unduplicated['case_id'] = c
    results.append(unduplicated)

## okay
So, *where* the unduplication step lives (and therefore where it would be accessible) is still up in the air.
* Should it live on the ActivityDataSet object? I kind of think so, but this ends up being a lot of shipping back and forth between types
* Should it be held within AnesthesiaDataSet stuff?
    * or, oh shit, should those have multiple inheritance with ActivityDataSet in order to gain access to unduplication methods?

In [ ]:
pd.concat(results)

In [ ]:
mwts = med_ds._with_timespans()
mwts['activity'] = 'medication'
mwts

In [ ]:
def check_end_of_concurrent_activity(row,
                                     activity_label='concurrent_activity_count',
                                     prev_label='prev'):
    if row[activity_label] == 0 and row[prev_label] > 0:
        return True
    else:
        return False


def check_start_of_concurrent_activity(row,
                                     activity_label='concurrent_activity_count',
                                     prev_label='prev'):
    if row[activity_label] > 0 and row[prev_label] <= 0:
        return True
    else:
        return False


from pytiva import ActivityDataSet
def unduplicated_ads_from_ads(ads, activities = None, activity_out_label = 'activity'):
    target = ads
    if activities is not None:
        target = target[ target.activity.isin(activities) ]
    
    ts = target.concurrency_ts()
    ts['prev'] = ts['concurrent_activity_count'].shift().fillna(0)
    
    is_start_col = 'is_start'
    is_end_col = 'is_end'
    ts[is_start_col] = ts.apply(func=check_start_of_concurrent_activity, axis=1)
    ts[is_end_col] = ts.apply(func=check_end_of_concurrent_activity, axis=1)
    
    paired_df = pd.DataFrame(
        [z for z in zip(ts[ ts[is_start_col] ].index, ts[ ts[is_end_col] ].index)],
        columns=['activity_start', 'activity_end']
    )
    
    paired_df['activity'] = activity_out_label
    
    return ActivityDataSet(paired_df)

def unduplicated_ads_from_med_ds(med_ds,
                                 
                                 td_offsets=(None, None)):
    
    pass

In [ ]:
#target_case_id = 201549596
med_activity_offsets = {
    'minutes_before_med': 2,
    'minutes_after_med': 2
}
target_case_id = 201549596

td_offset_before_med = pd.to_timedelta(med_activity_offsets['minutes_before_med'], unit='Min')
td_offset_after_med = pd.to_timedelta(med_activity_offsets['minutes_after_med'], unit='Min')
datetime_col = 'med_datetime'

df_meds_timespans = datetime_to_activity_timespan(
    df_meds_slim, datetime_col=datetime_col,
    td_before = td_offset_before_med,
    td_after = td_offset_after_med,
    inplace = False)

df_meds_timespans['activity'] = 'medication'

ads = ActivityDataSet(df_meds_timespans)#[ df_meds_timespans['case_id'] == target_case_id ])
ads

In [ ]:
bds = ads
bds = bds[ bds['case_id'] == target_case_id ]
bds

In [ ]:
ads

In [ ]:
unduplicated_ads_from_ads(ads)
#unduplicated_concurrent_case_activity(df_meds_timespans, activity_category='medication')

In [ ]:
ads.unduplicated_concurrency_ads()#activities=['medication'])

In [ ]:
med_activity_df = pd.DataFrame(paired, columns=['activity_start', 'activity_end'])
# TODO: set aside patient_id or case_id and pop it back in here
med_activity_df['patient_id'] = target_case_id
med_activity_df['activity'] = 'medication'
ds = pytiva.ActivityDataSet(med_activity_df)
ds

In [ ]:
# TODO: test that this is never greater than one--supposed to be unduplicated activity
ActivityDataSet(df_meds_slim[:10]).concurrency_ts()